 中文语言的文本分类技术和流程，主要包括以下几个步骤：
（1）预处理：去除文本的噪声信息，例如HTML标签，文本格式转换，检测句子边界等等；
（2）中文分词：使用中文分词器为文本分词，并去除停用词；
（3）构建词向量空间：统计文本词频，生成文本的词向量空间；
（4）权重策略--TF-IDF方法：使用TF-IDF发现特征词，并抽取为反映文档主题的特征；
（5）分类器：使用算法训练分类器；
（6）评价分类结果：分类器的测试结果分析。

In [5]:
########################################################################################
# 中文分词
import os
import jieba

# 定义保存文件函数
def savefile(file, savepath):
    f = open(savepath,'wb')
    f.write(file)
    f.close()
    pass

# 定义读取文件函数
def readfile(filename):
    fp = open(filename,'rb')
    contents = fp.read()
    fp.close()
    return contents

corpuses_path = "corpuses/"
corpuses_path_seg = "corpuses_seg/"

# 获取类别名称（）
category_list = os.listdir(corpuses_path)

for category in category_list:
    category_path = corpuses_path + category + '/'         # 每一个类别的路径
    category_path_seg = corpuses_path_seg + category + '/' # 拼出分词后每个类的存放路径
    if not os.path.exists(category_path_seg):
        os.makedirs(category_path_seg)
    
    file_name = os.listdir(category_path)                  # 获取每一类中的所有文件名
    
    for each_file_name in file_name:
        full_name = category_path + each_file_name         # 文件的全路径
        
        contents = readfile(full_name).strip()            # 文件的内容
        
        # 将多余空行替换为空
        contents = contents.replace('\r\n'.encode('utf-8'),"".encode('utf-8'))
        
        # 使用结巴分词
        contents_seg = jieba.cut(contents)
        
        # 保存分词后的文件内容
        savefile(" ".join(contents_seg).encode('utf-8'),category_path_seg + each_file_name)
    
print("中文分词结束！")

中文分词结束！


In [6]:
####################################################################################
# 使用 picplk 模块使数据的持久化
# 引入sklearn 的 Bunch 数据结构
from sklearn.datasets.base import Bunch
import os 
import pickle

def savefile(file_contents,file_path):
    fp = open(file_path,'wb')
    fp.write(file_contents).encode('utf-8')
    fp.close()
    pass

def readfile(file_path):
    fp = open(file_path,'rb')
    contents = fp.read().decode('utf-8')
    fp.close()
    return contents

# Bunch类提供一种key,value的对象形式
# target_name:  所有分类集名称列表
# label:       每个文件的分类标签列表
# filepath:    文件路径
# contents:    分词后文件词向量形式
bunch = Bunch(target_name = [],label =[],filepath = [],contents = [])

corpuses_bag_path = 'corpuses_bag/corpuses_bag.dat'   # 持久化数据存储路径
corpuses_path_seg = "corpuses_seg/"                   # 分词后数据存放文件夹

# 获取类别名称（）
category_list = os.listdir(corpuses_path_seg)
# 类别名称，总共用共多少类
bunch.target_name.extend(category_list)

for category in category_list:
    category_path_seg = corpuses_path_seg + category + '/'
    
    file_list = os.listdir(category_path_seg)
    
    for each_file in file_list:
        full_name = category_path_seg + each_file
        bunch.label.append(category)
        bunch.filepath.append(full_name)
        bunch.contents.append(readfile(full_name).strip())
        
# 数据持久化
file_obj = open(corpuses_bag_path,'wb')
pickle.dump(bunch,file_obj)
file_obj.close()

print('target_name:',bunch.target_name)
print('label:',bunch.label[5:15],'文本语句数量：',len(bunch.label))
print('contents:',bunch.contents[1][1:10],'文本语句数量：',len(bunch.contents))
print('filepath:',bunch.filepath[1])
print("构建文本对象结束！！！")

target_name: ['C000008', 'C000010', 'C000013', 'C000014', 'C000016', 'C000020', 'C000022', 'C000023', 'C000024']
label: ['C000008', 'C000008', 'C000008', 'C000008', 'C000008', 'C000008', 'C000008', 'C000008', 'C000008', 'C000008'] 文本语句数量： 17910
contents: 券 通 ： 百联  文本语句数量： 17910
filepath: corpuses_seg/C000008/100.txt
构建文本对象结束！！！


In [22]:
########################################################################################################
# 权重策略：TF-IDF方法（Tf–idf term weighting） 进行特征抽取

# 词频（term frequency，TF）指的是某一个给定的词语在该文件中出现的频率。
# 逆向文件频率（inverse document frequency，IDF）是一个词语普遍重要性的度量。
# 某一特定词语的IDF，可以由总文件数目除以包含该词语之文件的数目，再将得到的商取对数得到。
# Tf表示词语频率，而tf-idf表示术语频率乘以逆文档频率：
# tf-idf(t,d) = tf(t,d) x idf(t)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
import os

# 定义读取文件函数
def readfile(filename):
    fp = open(filename,'rb')
    contents = fp.read().decode('utf-8')
    fp.close()
    return contents

# 读取停词
stop_words_path = 'corpuses_bag/hlt_stop_words.txt'
stop_words = readfile(stop_words_path).splitlines()

# 读取数据
corpuses_bag_path = 'corpuses_bag/corpuses_bag.dat'   # 持久化数据存储路径
fp = open(corpuses_bag_path,'rb')
corpuses_bag = pickle.load(fp)
fp.close()

# 数据划分
corpuses_bag_train,corpuses_bag_test,y_train,y_test = train_test_split(corpuses_bag.contents,corpuses_bag.label,test_size=0.2,train_size = 0.8)
print('训练数据大小',len(corpuses_bag_train),len(y_train))
print('测试数据大小',len(corpuses_bag_test),len(y_test))

# 训练数据tf-idf 特征抽取
vectorizer = TfidfVectorizer(stop_words = stop_words, sublinear_tf = True, max_df = 0.5)
X_train = vectorizer.fit_transform(corpuses_bag_train)

训练数据大小 14328 14328
测试数据大小 3582 3582


In [23]:
X_train.vocabulary = vectorizer.vocabulary_
print(list(X_train.vocabulary.items())[0:10])
# 测试数据tf-idf 特征抽取
X_test = vectorizer.transform(corpuses_bag_test)
print("训练集大小：",X_train.shape)

[('kc', 19081), ('135', 2247), ('国际', 87862), ('在线', 88675), ('消息', 162533), ('英国', 202120), ('防务', 230731), ('新闻', 137868), ('2006', 3745), ('报道', 128034)]
训练集大小： (14328, 243931)


In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,classification_report

MulNB = MultinomialNB(alpha = 0.0001)
MulNB.fit(X_train,y_train)
predicted = MulNB.predict(X_test)
print('测试集混淆矩阵:\n',confusion_matrix(y_test,predicted))
print("测试集准确率：",MulNB.score(X_test,y_test))
print('测试集分类报告：\n',classification_report(y_test,predicted))
print('训练集分类报告：\n',classification_report(y_train,MulNB.predict(X_train)))
print("训练集准确率：",MulNB.score(X_train,y_train))

测试集混淆矩阵:
 [[324  17  11   0   9   2   6  11   3]
 [  6 324   9   1  14   7   8  19   5]
 [  6   7 339   0   6   3  26   8   1]
 [  0   1   0 388   7   0   1   2   1]
 [  1   6   5   0 362   2   6  25   2]
 [  1   4   8   0   2 341  15  42   1]
 [  0   3   4   0   2   5 325  35   1]
 [  3   2   4   0  15   9  10 372   4]
 [  0   1   1   0   0   4   0  28 359]]
测试集准确率： 0.874930206588
测试集分类报告：
              precision    recall  f1-score   support

    C000008       0.95      0.85      0.90       383
    C000010       0.89      0.82      0.85       393
    C000013       0.89      0.86      0.87       396
    C000014       1.00      0.97      0.98       400
    C000016       0.87      0.89      0.88       409
    C000020       0.91      0.82      0.87       414
    C000022       0.82      0.87      0.84       375
    C000023       0.69      0.89      0.77       419
    C000024       0.95      0.91      0.93       393

avg / total       0.88      0.87      0.88      3582

训练集分类报告：
          

In [10]:
##########################################################################################################
###################    停用词
# 由于文本在存储为向量空间时，维度比较高。为节省存储空间和提高搜索效率，在文本分类之前会自动
# 过滤掉某些字或词，这些字或词即被称为停用词
# 从这个网址下载停用词表：http://www.threedweb.cn/thread-1294-1-1.html
# 读取停用词列表
# 读取文件 
# 1. 读取停用词表 
# 读取文件

# 定义读取文件函数
def readfile(filename):
    fp = open(filename,'rb')
    contents = fp.read().decode('utf-8')
    fp.close()
    return contents

stop_words_path = 'corpuses_bag/hlt_stop_words.txt'

#  按照行('\r', '\r\n', \n')分隔，返回一个包含各行作为元素的列表，
#  str.splitlines([keepends])如果参数 keepends 为 False，不包含换行符，如果为 True，则保留换行符。
stop_words = readfile(stop_words_path).splitlines()

print(stop_words[10:20])

['*', '一一', '~~~~', '’', '. ', '『', '.一', './', '-- ', '』']
